In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame

%matplotlib inline
pd.options.display.mpl_style = 'default'
import os
from IPython.core.display import HTML

In [3]:
YELP_DATACHALLENGE_URL = 'http://www.yelp.com/dataset_challenge'
tag = '<iframe src="{0}" width="1100px" height="333px">'.format(YELP_DATACHALLENGE_URL)
HTML(tag)

# Yelp Dataset
This link gives you an idea of the structure of Data Schema: all JSON files with each line as a distinct record
http://www.yelp.com/dataset_challenge
There are 5 tables:

  1. Business (42,153 records)

  2. Review (1,125,458 records)

  3. Check ins (31,617 records)

  4. Tips (403,210 records)

  5. Users (252,898 records)


In [4]:
json_raw_data_dir = "json_raw"
json_clean_data_dir =  "json_clean"
!ls -sh $json_raw_data_dir
!echo
!ls -sh $json_clean_data_dir

total 3,2G
 66M yelp_academic_dataset_business.json  229M yelp_academic_dataset_review_p1.json  228M yelp_academic_dataset_review_p4.json
 25M yelp_academic_dataset_checkin.json   245M yelp_academic_dataset_review_p2.json  114M yelp_academic_dataset_tip.json
1,9G yelp_academic_dataset_review.json	  220M yelp_academic_dataset_review_p3.json  226M yelp_academic_dataset_user.json

total 3,2G
 66M yelp_academic_dataset_business.json  229M yelp_academic_dataset_review_p1.json  229M yelp_academic_dataset_review_p4.json
 25M yelp_academic_dataset_checkin.json   246M yelp_academic_dataset_review_p2.json  114M yelp_academic_dataset_tip.json
1,9G yelp_academic_dataset_review.json	  220M yelp_academic_dataset_review_p3.json  226M yelp_academic_dataset_user.json


## Loading files from the clean data directory

In [5]:
business_json_filename = os.path.join(json_clean_data_dir, 'yelp_academic_dataset_business.json')
business_DF = pd.io.json.read_json(business_json_filename)
business_DF.rename(columns={'name': 'business_name', 'review_count': 'business_review_count', 'stars': 'business_stars'}, inplace=True)

In [6]:
user_json_filename = os.path.join(json_clean_data_dir, 'yelp_academic_dataset_user.json')
user_DF = pd.io.json.read_json(user_json_filename)
user_DF.rename(columns={'name': 'user_name', 'review_count': 'user_review_count', 'votes': 'user_votes'}, inplace=True)

In [7]:
review_json_filename = os.path.join(json_clean_data_dir, 'yelp_academic_dataset_review_p1.json')
review_DF = pd.io.json.read_json(review_json_filename)
review_DF.rename(columns={'stars': 'review_stars', 'text': 'review_text', 'date': 'review_date', 'votes': 'review_votes'}, inplace=True)

In [9]:
tip_json_filename = os.path.join(json_clean_data_dir, 'yelp_academic_dataset_tip.json')
tip_DF = pd.io.json.read_json(tip_json_filename)
tip_DF.rename(columns={'text': 'tip_text', 'date': 'tip_date'}, inplace=True)

## Saving as CSV

In [10]:
business_csv_filename = os.path.join("yelp_csv", 'yelp_academic_dataset_business.csv')
user_csv_filename = os.path.join("yelp_csv", 'yelp_academic_dataset_user.csv')

if not os.path.isfile(business_csv_filename):
    business_DF.to_csv(business_csv_filename, index=False, encoding='utf-8')
    
if not os.path.isfile(user_csv_filename):
    user_DF.to_csv(user_csv_filename, index=False, encoding='utf-8')

## Merging DataTables/Frames

In [12]:
business_and_review_DF = pd.merge(business_DF, review_DF, on='business_id', how='inner')
business_and_user_and_review_DF = pd.merge(business_and_review_DF, user_DF, on ='user_id', how='inner')

In [14]:
all_DF = pd.merge(business_and_user_and_review_DF, tip_DF, on=['user_id', 'business_id'], how='inner')
all_DF.head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,business_name,neighborhoods,...,friends,user_name,user_review_count,type_x,user_votes,yelping_since,tip_date,likes,tip_text,type_y
0,{},6c2pWWtdQZA3w5raX6k-dg,"[Barbers, Beauty & Spas]",Homestead,"3510 Main St\nHomestead, PA 15120","{u'Tuesday': {u'close': u'17:30', u'open': u'0...",40.385508,-79.903491,Carmine's Barber Shop,[],...,"[DsN2dJ6SnTEACr1qC5zpGg, UD7Y1CqfY6mDmRwIuCf6n...",Kevin,47,user,"{u'funny': 8, u'useful': 46, u'cool': 10}",2011-11,2014-07-14,0,I've been told that they do open at 4 AM and i...,tip
1,"{u'Take-out': True, u'Price Range': 2, u'Outdo...",1qCuOcks5HRv67OHovAVpg,"[Pubs, Bars, American (New), Nightlife, Restau...",Homestead,"122 W 8th Ave\nHomestead\nHomestead, PA 15120",{},40.405311,-79.912595,Duke's Upper Deck Cafe,[Homestead],...,"[fwsJGulnozT2U6FefsLiFw, Dsh4UCo9ny0XCrEJaF4aK...",Molly,146,user,"{u'funny': 36, u'useful': 136, u'cool': 70}",2012-06,2014-10-18,0,Half priced appetizers after 9 on week nights.,tip
2,"{u'Alcohol': u'beer_and_wine', u'Open 24 Hours...",2X5G4Ujq0s4Wfn4TC7gX0g,"[Pubs, Bars, Hot Dogs, Nightlife, Restaurants]",Pittsburgh,"1118 S Braddock Ave\nPittsburgh, PA 15218","{u'Monday': {u'close': u'23:00', u'open': u'11...",40.432022,-79.893834,D's Six Pax & Dogz,[],...,"[fwsJGulnozT2U6FefsLiFw, Dsh4UCo9ny0XCrEJaF4aK...",Molly,146,user,"{u'funny': 36, u'useful': 136, u'cool': 70}",2012-06,2014-10-18,0,The special hummus is always good. Take a brea...,tip
3,{u'Good for Kids': True},cE27W9VPgO88Qxe4ol6y_g,"[Active Life, Mini Golf, Golf]",Bethel Park,"1530 Hamilton Rd\nBethel Park, PA 15234",{},40.354115,-80.014660,Cool Springs Golf Center,[],...,"[nEYPahVwXGD2Pjvgkm7QqQ, ukf5AhPo3dJhEWB2km-St...",Andrew,29,user,"{u'funny': 18, u'useful': 61, u'cool': 11}",2012-11,2013-04-18,0,Don't waste your time.,tip
4,"{u'Take-out': True, u'Accepts Credit Cards': T...",McikHxxEqZ2X0joaRNKlaw,"[Food, American (Traditional), Breweries, Rest...",Homestead,"171 E Bridge St\nHomestead\nHomestead, PA 15120","{u'Monday': {u'close': u'02:00', u'open': u'11...",40.409435,-79.915262,Rock Bottom,[Homestead],...,"[D3JoaaL9kEgnIs64SUOR2w, FP79xn8H2XXnMksRuP5l6...",Casey,107,user,"{u'funny': 49, u'useful': 202, u'cool': 141}",2011-03,2015-03-28,0,Longest wait of all time on Saturdays.,tip


## Get rid of finished stuff

In [15]:
%xdel business_and_review_DF

In [16]:
%reset_selective business_and_user_and_review_DF

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [17]:
%reset_selective business_DF
%reset_selective user_DF
%reset_selective review_DF
%reset_selective tip_DF

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y
Once deleted, variables cannot be recovered. Proceed (y/[n])?  y
Once deleted, variables cannot be recovered. Proceed (y/[n])?  y
Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


## Save merged data frame

In [18]:
all_csv_filename = os.path.join("yelp_csv", 'all_p1.csv')

if not os.path.isfile(all_csv_filename):
    all_DF.to_csv(all_csv_filename, index=False, encoding='utf-8')
else:
    all_DF = pd.read_csv(all_csv_filename)
    
!ls -sh $all_csv_filename

91M yelp_csv/all_p1.csv


In [19]:
print all_DF.columns
all_DF.describe()

Index([           u'attributes',           u'business_id',
                  u'categories',                  u'city',
                u'full_address',                 u'hours',
                    u'latitude',             u'longitude',
               u'business_name',         u'neighborhoods',
                        u'open', u'business_review_count',
              u'business_stars',                 u'state',
                      u'type_x',           u'review_date',
                   u'review_id',          u'review_stars',
                 u'review_text',                u'type_y',
                     u'user_id',          u'review_votes',
               u'average_stars',           u'compliments',
                       u'elite',                  u'fans',
                     u'friends',             u'user_name',
           u'user_review_count',                u'type_x',
                  u'user_votes',         u'yelping_since',
                    u'tip_date',                 u'likes

,latitude,longitude,open,business_review_count,business_stars,review_stars,average_stars,fans,user_review_count,likes
count,32278.000000,32278.000000,32278,32278.000000,32278.000000,32278.000000,32278.000000,32278.000000,32278.000000,32278.000000
mean,35.418011,-104.756164,0.929023,188.429426,3.615481,3.970351,3.846797,14.285705,177.615032,0.012671
std,2.558386,13.921216,0.256791,333.781377,0.647965,1.199325,0.559875,48.573419,319.396947,0.127392
min,32.877390,-115.327837,False,3.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000
25%,33.493593,-115.043175,1,24.000000,3.000000,3.000000,3.620000,0.000000,19.000000,0.000000
50%,35.094835,-111.979599,1,74.000000,3.500000,4.000000,3.860000,2.000000,76.000000,0.000000
75%,36.142636,-89.390696,1,187.000000,4.000000,5.000000,4.130000,9.000000,196.000000,0.000000
max,43.275562,-79.850855,True,1907.000000,5.000000,5.000000,5.000000,1323.000000,5648.000000,3.000000
